In [1]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import SimpleRNN, TimeDistributed
from sklearn import model_selection

csv_data=pd.read_csv("data/blader-data.csv")

Using TensorFlow backend.


In [2]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [3]:
all_features=csv_data.iloc[:, 0:22]
all_labels=csv_data['qclass']

encoder.fit(all_labels)
all_labels_encoded = encoder.transform(all_labels)

In [4]:
X_train,X_test,Y_train,Y_test=model_selection.train_test_split(all_features,all_labels_encoded,test_size=0.30)

In [5]:
print(Y_test.sum())

255


In [6]:
input_dim=X_train.shape[1]
print(input_dim)

22


In [7]:
#do SMOTE over-sampling
from imblearn.over_sampling import SMOTE
sm=SMOTE()
X_train_sampled, Y_train_sampled = sm.fit_sample(X_train, Y_train)

In [8]:
print(Y_train_sampled)

[0 1 1 ... 1 1 1]


In [9]:
# categorial encoding of the class labels
train_one_hot_labels = keras.utils.to_categorical(Y_train_sampled, num_classes=2)
test_one_hot_labels=keras.utils.to_categorical(Y_test, num_classes=2)



In [10]:
#model creation

model = Sequential()
model.add(Dense(64,input_shape=(input_dim,)))
model.add(Activation("tanh"))
model.add(Dense(output_dim=2))
model.add(Activation("softmax"))

model.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                1472      
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 1,602
Trainable params: 1,602
Non-trainable params: 0
_________________________________________________________________


c:\logiciels\python-3.7.3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=2)`
  


In [19]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [12]:
model.fit(X_train_sampled, train_one_hot_labels, epochs=822, verbose=0)

In [13]:
predictions=model.predict(X_test)


from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(test_one_hot_labels.argmax(axis=1), predictions.argmax(axis=1), labels=[0,1])
print(matrix)

[[451 284]
 [144 111]]


In [14]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    
    # Begin CHANGES
    fst_empty_cell = (columnwidth-3)//2 * " " + "t/p" + (columnwidth-3)//2 * " "
    
    if len(fst_empty_cell) < len(empty_cell):
        fst_empty_cell = " " * (len(empty_cell) - len(fst_empty_cell)) + fst_empty_cell
    # Print header
    print("    " + fst_empty_cell, end=" ")
    # End CHANGES
    
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
        
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [15]:
print_cm(matrix, ["0","1"])

     t/p      0     1 
        0 451.0 284.0 
        1 144.0 111.0 


In [16]:
print(Y_train_sampled.sum(), len(Y_train_sampled))

1740 3480


In [17]:
print(Y_test.sum(), len(Y_test)-Y_test.sum())

255 735
